In [1]:
# CONFIG
import os
from dotenv import load_dotenv

load_dotenv()

# Airtable configuration
AIRTABLE_API_KEY = os.getenv('AIRTABLE_API_KEY')
AIRTABLE_BASE_ID = os.getenv('AIRTABLE_BASE_ID')
AIRTABLE_CD_TABLE = os.getenv('AIRTABLE_CD_TABLE')

# Data Retrieval

In [27]:
import pandas as pd
from pyairtable import Api

# data processing
def remove_fields_prefix(df):
    df.columns = df.columns.str.replace('fields.', '')
    return df

def process_list_like_values(df):
    for col in df.columns:
        if df[col].apply(lambda x: isinstance(x, list)).any():
            df[col] = df[col].apply(lambda x: ','.join(map(str, x)) if isinstance(x, list) else x)
    return df

# data fetching
def fetch_data(table_id):
    api = Api(AIRTABLE_API_KEY)
    table = api.table(AIRTABLE_BASE_ID, table_id)
    records = table.all()

    if records:
        df = pd.json_normalize([record['fields'] for record in records])  # Directly normalize 'fields'
        df = remove_fields_prefix(df)
        df = process_list_like_values(df)
        return df
    else:
        return pd.DataFrame()  # Return an empty DataFrame if no records

# implementation
df = fetch_data(AIRTABLE_CD_TABLE)

columns_to_check = [
    "Relevant facts / Summary of the case", 
    "Relevant rules of law involved", 
    "Choice of law issue", 
    "Court's position"
]

df = df.dropna(subset=columns_to_check)

cases = df['Case'].to_list()
abstracts = df['Abstract'].to_list()
texts = df['Content'].to_list()
facts = df['Relevant facts / Summary of the case'].to_list()
rules = df['Relevant rules of law involved'].to_list()
issues = df['Choice of law issue'].to_list()
positions = df["Court's position"].to_list()

# Case Analysis

In [ ]:
# utils
import subprocess
import json

def prompt_llama(prompt_text):
    # Run the Ollama command to prompt LLaMA and capture output
    process = subprocess.run(
        ["ollama", "generate", "llama3.1", "--input", prompt_text],
        capture_output=True,
        text=True
    )

    # Handle errors if they occur
    if process.returncode != 0:
        print("Error:", process.stderr)
        return None
    
    # Parse the output
    output = process.stdout.strip()  # Adjust based on the format returned by the model
    return output

## 0. Abstract

In [28]:
# extract abstracts from texts
prompt_text = "What is the capital of Switzerland?"
model_output = prompt_llama(prompt_text)

## 1. Relevant Facts

In [ ]:
# relevant facts

## 2. Relevant Rules of Law

In [ ]:
# relevant rules of law

## 3. Choice of Law Issue

In [ ]:
# choice of law issue

## 4. Court's Position

In [ ]:
# court's position